Install requirements.

In [1]:
!pip install -r requirements.txt -qq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.150 requires docutils<0.17,>=0.10, but you have docutils 0.19 which is incompatible.


Download a wandb model below.

In [2]:
import wandb

# define wandb environment
%env WANDB_LOG_MODEL=true
%env WANDB_API_KEY=53c099408fab02d1e4fff7386e8dfc1e759689a1

env: WANDB_LOG_MODEL=true
env: WANDB_API_KEY=53c099408fab02d1e4fff7386e8dfc1e759689a1


In [10]:
# download wandb model
run = wandb.init()
artifact = run.use_artifact('oumar-kane-team/t5-small-custom-train-wf-translation-bayes-hpsearch-v5_2/model_nufibzx:v0', type='dataset')
artifact_dir = artifact.download()

wandb: Downloading large artifact model_nufibzx:v0, 176.03MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:4.8


In [11]:
wandb.finish()

In [12]:
# transfer the model inside the artifact to data/checkpoints/name_of_model
def transfer_model(artifact_dir, model_name):
    import os
    import shutil
    import glob

    # transfer the model inside the artifact to data/checkpoints/name_of_model
    os.makedirs(model_name, exist_ok=True)
    for file in glob.glob(f'{artifact_dir}/*'):
        shutil.copy(file, model_name)

    # delete the artifact
    shutil.rmtree(artifact_dir)

# transfer the model inside the artifact to name_of_model
model_name = 'wf_t5_small_custom_train_v5_2_checkpoints'
transfer_model(artifact_dir, model_name)

In [13]:
import os
os.listdir(model_name)

['checkpoints.pth', 'checkpoints.json']

In [14]:
os.rename(f'{model_name}/checkpoints.json', f'{model_name}/best_checkpoints.json')
os.rename(f'{model_name}/checkpoints.pth', f'{model_name}/best_checkpoints.pth')

-----------------

Send a directory as a wandb artifact.

In [8]:
# initialize a project and add a directory as artifact to wandb 
artifact_name = 't5_small_custom_train_results_fw_v5_2'
def add_directory(directory, artifact_name, project: str = 'fw_artifacts', entity: str = 'oumar-kane-team'):
    
    run = wandb.init(project=project, entity=entity)
    
    # add a directory as artifact to wandb
    artifact = wandb.Artifact(artifact_name, type='dataset')
    artifact.add_dir(directory)
    run.log_artifact(artifact)
    
    wandb.finish()
    
add_directory(artifact_name, artifact_name)

wandb: Adding directory to artifact (./t5_small_custom_train_results_fw_v5_2)... Done. 0.5s


Compress the logs into an archive.

In [9]:
import shutil

logs_path = 'data/logs/t5_small_custom_train_fw'

file = 'version_5_2'

shutil.make_archive('logs', 'zip', logs_path, file)

'/home/ec2-user/SageMaker/en_us/Wolof_traduction/logs.zip'

Show the current device.

In [12]:
import torch

In [13]:
torch.cuda.get_device_name()

'Tesla V100-SXM2-16GB'